In [1]:
import pandas as pd

In [4]:
data = pd.read_csv("./history_data.csv")


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32659 entries, 0 to 32658
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DateTime  32659 non-null  object 
 1   AQI       32659 non-null  int64  
 2   CO        32659 non-null  float64
 3   NO        32659 non-null  float64
 4   NO2       32659 non-null  float64
 5   O3        32659 non-null  float64
 6   SO2       32659 non-null  float64
 7   PM2.5     32659 non-null  float64
 8   PM10      32659 non-null  float64
 9   NH3       32659 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 2.5+ MB


In [7]:
!pip3 install pandas numpy scikit-learn matplotlib tensorflow

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
    

In [10]:

# Load the dataset (replace the path with your actual path)
data = pd.read_csv('./data/history_data.csv', parse_dates=['DateTime'], index_col='DateTime')

# Select relevant pollutant features
pollutant_features = ['CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10', 'NH3']

# Normalize the pollutant features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[pollutant_features])

# Function to create time series windows
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

SEQ_LENGTH = 60  # Use past 60 hours (or adjust based on experiment)

# Create sequences
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], len(pollutant_features)))

# Split data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
    

In [11]:

# Build the LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(SEQ_LENGTH, len(pollutant_features))))
model.add(LSTM(100))
model.add(Dense(50))
model.add(Dense(len(pollutant_features)))  # Output pollutant values for next hour

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))
    

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - loss: 0.0146 - val_loss: 0.0019
Epoch 2/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - loss: 0.0018 - val_loss: 9.4701e-04
Epoch 3/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 43ms/step - loss: 9.1565e-04 - val_loss: 7.0662e-04
Epoch 4/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - loss: 6.5148e-04 - val_loss: 8.3882e-04
Epoch 5/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - loss: 5.7856e-04 - val_loss: 6.7891e-04
Epoch 6/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 43ms/step - loss: 5.2589e-04 - val_loss: 5.6302e-04
Epoch 7/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - loss: 4.9402e-04 - val_loss: 5.1990e-04
Epoch 8/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - loss: 4.7280e-04 - val_loss: 5.1388e-04
Epoch 9/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - loss: 4.2966e-04 - val_loss: 5.4958e-04
Epoch 10/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - loss: 4.2463e-04 - val_loss: 4.6175e-04


In [20]:
X_test[-1]

array([[0.04259755, 0.08074651, 0.97463768, 0.93091046, 0.19644685,
        0.02289614, 0.91572673, 0.07409657],
       [0.03585007, 0.05900465, 0.97450443, 0.9311562 , 0.16855534,
        0.02025249, 0.91539251, 0.06635287],
       [0.03585007, 0.05755938, 0.97448789, 0.93138984, 0.16332797,
        0.02017096, 0.91534868, 0.06544185],
       [0.03585007, 0.04216413, 0.97462115, 0.93118133, 0.14589121,
        0.01964689, 0.9153432 , 0.06544185],
       [0.03585007, 0.03933643, 0.97452096, 0.93112734, 0.13192718,
        0.01927422, 0.91538429, 0.06437899],
       [0.04006788, 0.04216413, 0.97460461, 0.9309812 , 0.12669981,
        0.01955372, 0.91544456, 0.06544185],
       [0.0535641 , 0.05617695, 0.9747048 , 0.93081923, 0.12498172,
        0.02410733, 0.91602076, 0.0856362 ],
       [0.07549592, 0.07377152, 0.97473787, 0.93076431, 0.12322708,
        0.03301656, 0.91704076, 0.11342241],
       [0.08730527, 0.07025261, 0.97473787, 0.93076617, 0.11799971,
        0.03929378, 0.917774

In [22]:
from aqi_cal import ret_future, ret

In [28]:
# Assuming you have the ret() function that fetches real-time data
def fetch_current_pollutant_data():
    # This calls your ret() function to get the current pollutants
    api_data = ret()
    
    # Extract pollutant values from the API response (without 'NO')
    current_pollutants = [
        api_data['pollutants']['CO'],       # CO
        0,                                  # Placeholder for 'NO'
        api_data['pollutants']['NO2'],      # NO2
        api_data['pollutants']['O3'],       # Ozone (O3)
        api_data['pollutants']['SO2'],      # SO2
        api_data['pollutants']['PM2.5'],    # PM2.5
        api_data['pollutants']['PM10'],     # PM10
        api_data['pollutants']['NH3']       # NH3
    ]
    
    # Normalize the current pollutant values using the same scaler used for training
    current_pollutants_scaled = scaler.transform([current_pollutants])
    
    # Return the current values in the proper format for LSTM input
    return current_pollutants_scaled


# Forecasting Function (with API current data as input)
def forecast_next_hours_with_api(model, current_values, scaler, forecast_hours=8):
    predictions = []
    current_input = current_values.copy()  # Start with current values
    
    for _ in range(forecast_hours):
        # Reshape input to [1, SEQ_LENGTH, number_of_features]
        current_input = np.reshape(current_input, (1, SEQ_LENGTH, len(pollutant_features)))
        
        # Predict the next hour pollutants
        predicted_pollutants = model.predict(current_input)
        
        # Inverse transform the predicted pollutant values to their original scale
        predicted_pollutants = scaler.inverse_transform(predicted_pollutants)
        
        # Store predictions
        predictions.append(predicted_pollutants)
        
        # Update input (shift the window forward and append new predictions)
        current_input = np.append(current_input[:, 1:, :], predicted_pollutants.reshape(1, 1, len(pollutant_features)), axis=1)
    
    return np.array(predictions).reshape(forecast_hours, len(pollutant_features))

# Fetch current pollutant data from the API
current_pollutants_scaled = fetch_current_pollutant_data()

# Forecast pollutant levels for the next 8 hours based on the current values from the API
predicted_pollutants = forecast_next_hours_with_api(model, current_pollutants_scaled, scaler)

predicted_pollutants  # This will give you the predicted pollutant values for the next 8 hours


{'calulated_values': {'PM10': '39.38', 'PM2.5': '16.93', 'SO2': '2.32', 'NO2': '5.45', 'CO': '8.77', 'O3': '1.38', 'Temperature': '26.88', 'Humidity': '82.47', 'Heat Index': '92.25', 'Relative Humidity': '79.24', 'Pressure': '1,007.05', 'Wind Speed': '3.53'}, 'top_3_pollutants': [{'pollutant': 'PM10', 'average': '39.38', 'industry_strategies': ['Adopt Clean Technologies', 'Improve Emission Standards', 'Dust Control Measures'], 'personal_strategies': ['Use Air Purifiers', 'Wear Masks', 'Avoid Outdoor Activities']}, {'pollutant': 'PM2.5', 'average': '16.93', 'industry_strategies': ['Enhance Air Filtration Systems', 'Switch to Cleaner Fuels', 'Implement Emission Controls'], 'personal_strategies': ['Limit Outdoor Activities', 'Use Air Purifiers', 'Wear N95 Masks']}, {'pollutant': 'CO', 'average': '8.77', 'industry_strategies': ['Engine Tuning and Maintenance', 'Promote Electric Vehicles', 'Improve Combustion Efficiency'], 'personal_strategies': ['Install CO Detectors', 'Regular Vehicle Mai

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: cannot reshape array of size 8 into shape (1,60,8)

In [14]:
# Example modified Step 5 for calculating AQI with custom function

# Define the ret_future() function using your pollutant forecasting logic
def calculate_future_aqi(predicted_pollutants, time_step):
    # Extract individual pollutants from the predicted pollutant array
    pm2_5 = predicted_pollutants[5]  # PM2.5
    pm10 = predicted_pollutants[6]   # PM10
    no2 = predicted_pollutants[2]    # NO2
    co = predicted_pollutants[0]     # CO
    so2 = predicted_pollutants[4]    # SO2

    # Calculate AQI using the custom ret_future function
    future_aqi = ret_future(pm2_5, pm10, no2, co, so2, time_step)
    
    return future_aqi

# Calculate AQI for each of the next 8 hours based on the predicted pollutant values
aqi_forecast = [calculate_future_aqi(predicted_pollutants[i], i+1) for i in range(8)]

# Show AQI forecast with remarks and impact for each hour
for i, forecast in enumerate(aqi_forecast):
    print(f"Hour {i+1}: AQI = {forecast['aqi']}, Remark = {forecast['remark']}, Impact = {forecast['impact']}")


Hour 1: AQI = 4.017, Remark = Good, Impact = Minimal impact
Hour 2: AQI = 3.57, Remark = Good, Impact = Minimal impact
Hour 3: AQI = 3.205, Remark = Good, Impact = Minimal impact
Hour 4: AQI = 2.955, Remark = Good, Impact = Minimal impact
Hour 5: AQI = 2.844, Remark = Good, Impact = Minimal impact
Hour 6: AQI = 2.852, Remark = Good, Impact = Minimal impact
Hour 7: AQI = 2.934, Remark = Good, Impact = Minimal impact
Hour 8: AQI = 3.039, Remark = Good, Impact = Minimal impact
